# Проект по анализу оттока клиентов банка
## Описание задачи проекта
Из «Бета-Банка» каждый месяц немного стали уходить клиенты. По данным банковских маркетологов сохранять текущих клиентов дешевле, чем привлекать новых.
Необходимо спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Для проекта предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 




## Подготовка и получение данных

In [1]:
import pandas as pd 
import seaborn as sns
import random 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

In [2]:
#читаем файл с компа или с сайта
try:
    df = pd.read_csv('/home/dmitrii/Документы/ucheba/Churn.csv')
except:
    df = pd.read_csv('C:/Users/Dmitrii/Documents/Dstudy/datasets/Churn.csv')

Рассмотрим первые 5 строчек датафрейма и его параметры. 

In [3]:
print(df.head(),'\n')
df.info()

   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0     2.0       0.00              1          1               1   
1     1.0   83807.86              1          0               1   
2     8.0  159660.80              3          1               0   
3     1.0       0.00              2          0               0   
4     2.0  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         790

## Исследование задачи

10000 записей в датасете достаточно для обучения, но в столбце "Tenure" отсутствует около 1000 записей. Ввиду того, что эта информация непосредственно не должна влиять на результирующую переменную и маловероятно, что распределена нормально, то разумно заполнить пустые значения случайными значениями от 0 до 10.

In [4]:
#заполним пустые значения столбца Tenure
df['Tenure'].fillna( random.randint(0,10), inplace=True )

Т.к. алгоритм должен предсказывать поведение клиентов в будущем, а не в прошлом, то личная информация прошлых клиентов не несет никакой пользы для модели. Удалим столбцы RowNumber, CustomerId, Surname.

In [5]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'],axis=1)

Просматривая типы данных видно, что столбцы Geography и Gender необходимо закодировать. Пол легко можно закодировать по бинарному принципу одним столбцом, т.к. столбец содержит только значения Female и Male.

In [6]:
df['Gender'] = pd.get_dummies(df['Gender'], drop_first=True)

Страну закодируем присвоением порядкового номера категории с помощью OrdinalEncoder.

In [7]:
encoder = OrdinalEncoder()
data_ordinal = pd.DataFrame(encoder.fit_transform(df),
                            columns=df.columns)
print(data_ordinal.head(5))

   CreditScore  Geography  Gender   Age  Tenure  Balance  NumOfProducts  \
0        228.0        0.0     0.0  24.0     2.0      0.0            0.0   
1        217.0        2.0     0.0  23.0     1.0    743.0            0.0   
2        111.0        0.0     0.0  24.0     8.0   5793.0            2.0   
3        308.0        0.0     0.0  21.0     1.0      0.0            1.0   
4        459.0        2.0     0.0  25.0     2.0   3696.0            0.0   

   HasCrCard  IsActiveMember  EstimatedSalary  Exited  
0        1.0             1.0           5068.0     1.0  
1        0.0             1.0           5639.0     0.0  
2        1.0             0.0           5707.0     1.0  
3        0.0             0.0           4704.0     0.0  
4        1.0             1.0           3925.0     0.0  


По таблице видно, что ряд столбцов имеют очент высокие абсолютные значения, что негативно сказывается для алгоритмов обучения. Однако перед стандартизацией значений необходимо разделить датасет на две части для исключения влияния функции стандартизации на тестовые данные, т.к. при стандартизации происходит деление на дисперсию и вычитание среднего всего датасета, а не только тестовой части. 

## Разделение датасейта
Разделим датасет на три части - целевую и признаков. 

In [8]:
#Разделение на целевую и выборку с признаками
target = data_ordinal['Exited']
features = data_ordinal.drop('Exited', axis=1)
#Разделение на обучающие и выборки для оценки
features_train, features_var, target_train,target_var = train_test_split(
    features, 
    target, 
    test_size=0.4, 
    random_state=12345)
#Разделение на валидационную и тествую выборки
features_valid, features_test, target_valid,target_test = train_test_split(
    features_var, 
    target_var, 
    test_size=0.5, 
    random_state=12345)

## Стандартизация значений
Стандартизацию значений произведем при помощи метода StandardScaler(). Обработке подвергаются столбцы 'CreditScore', 'Age', 'Balance', 'Age','Tenure' и 'EstimatedSalary'.

In [9]:
#Столбцы для стандартизации
numeric = ['CreditScore', 'Age', 'Balance', 'Age','Tenure', 'EstimatedSalary']
#Инициализируем стандартизатор
scaler = StandardScaler()
#Обучаем стандартизатор на тренировочных данных
scaler.fit(features_train[numeric])
#Стандартизируем тренировочную и валидационную выборки
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])
print(features_train.head())

      CreditScore  Geography  Gender       Age    Tenure   Balance  \
7479    -0.888356        2.0     1.0 -0.373277  0.810280  1.663957   
3411     0.608996        0.0     0.0 -0.183370  0.810280  0.309701   
6027     2.054357        1.0     1.0  0.481302 -0.802217  1.332696   
1247    -1.460262        0.0     1.0 -1.417761  0.165281 -0.956910   
3716     0.130675        1.0     0.0 -1.132902 -1.124717  1.529191   

      NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  
7479            0.0        1.0             0.0        -0.190775  
3411            0.0        0.0             0.0        -0.335352  
6027            1.0        0.0             1.0         1.501017  
1247            1.0        1.0             0.0        -1.075144  
3716            0.0        0.0             0.0         1.515855  


## Тестирование модели

### Обучение несбалансированной выборки

Посмотрим на балансировку класcов целевого параметра. Имеется явный перекос почти в 4 раза в пользу оставшихся клиентов, что является благом для банка, но не для модели обучения. Однако попытаемся обучить такую модель.   

In [10]:
print(df.groupby('Exited')['Exited'].count())

Exited
0    7963
1    2037
Name: Exited, dtype: int64


### Обучение модели случайного леса

In [11]:
# лучшие метрики
f1_best = 0
roc_auc_best=0
# лучшие параметры случайного леса
n_est_best = 0
n_depth_best = 0

for depth in range(6, 15, 3): 
    for est in range(300, 1300, 300):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = round( f1_score(target_valid, predicted_valid),4)
        #roc_auc вычисление
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid),4)
        print("Кол-во деревьев:",est ,"  глубина:", depth,"  F1:", f1, "  ROC_AUC:", roc_auc)
        
        #ищем лучшую модель по количеству деревьев
        if f1>f1_best and roc_auc>roc_auc_best:
            f1_best = f1
            roc_auc_best = roc_auc
            n_est_best = est
            n_depth_best = depth
            model_fr_best = model

probabilities_valid = model_fr_best.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid),4)

print('-лучший результат-')
print("Кол-во деревьев:",n_est_best,"  глубина:", n_depth_best,"  F1:", f1_best , "  ROC_AUC:", roc_auc_best)

Кол-во деревьев: 300   глубина: 6   F1: 0.5342   ROC_AUC: 0.8489
Кол-во деревьев: 600   глубина: 6   F1: 0.5375   ROC_AUC: 0.8494
Кол-во деревьев: 900   глубина: 6   F1: 0.5325   ROC_AUC: 0.8494
Кол-во деревьев: 1200   глубина: 6   F1: 0.5357   ROC_AUC: 0.8491
Кол-во деревьев: 300   глубина: 9   F1: 0.5638   ROC_AUC: 0.8507
Кол-во деревьев: 600   глубина: 9   F1: 0.5651   ROC_AUC: 0.8511
Кол-во деревьев: 900   глубина: 9   F1: 0.5647   ROC_AUC: 0.8516
Кол-во деревьев: 1200   глубина: 9   F1: 0.5638   ROC_AUC: 0.8515
Кол-во деревьев: 300   глубина: 12   F1: 0.5652   ROC_AUC: 0.8488
Кол-во деревьев: 600   глубина: 12   F1: 0.5639   ROC_AUC: 0.8478
Кол-во деревьев: 900   глубина: 12   F1: 0.563   ROC_AUC: 0.8482
Кол-во деревьев: 1200   глубина: 12   F1: 0.5616   ROC_AUC: 0.8474
-лучший результат-
Кол-во деревьев: 600   глубина: 9   F1: 0.5651   ROC_AUC: 0.8511


Результат получился ниже требуемого заданием. Вероятнее всего сыграла негативную роль разбалансировка классов. Лучшие результаты в ряду деревьев 300, 600, 900, 1200 и глубине 6,9,12 показала модель с гиперпараметрами 900 деревьев и максимальной глубиной 9.

### Обучение модели логистической регрессии
Для сравнения обучим модель логистической регрессии 

In [12]:
# лучшие метрики
f1_best = 0
roc_auc_best=0
# лучшие параметры логистической регрессии
n_iter_best = 0

for n_iter in range(500, 200000, 20000): 
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=n_iter)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = round( f1_score(target_valid, predicted_valid),4)
    #вычисление roc_auc
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid),4)
    print("Кол-во итераций:", n_iter,"  F1:", f1, "  ROC_AUC:", roc_auc)
        
    #ищем лучшую модель по количеству деревьев
    if f1>f1_best and roc_auc>roc_auc_best:
            f1_best = f1
            roc_auc_best = roc_auc
            n_iter_best = n_iter
            model_log_best = model

predicted_valid = model_log_best.predict(features_valid)
f1 = round( f1_score(target_valid, predicted_valid), 4)
#получаем вероятности признака 1
probabilities_valid = model_log_best.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid),4)
print('-лучший результат-')
print("Кол-во итераций:",n_iter_best,"  F1:", f1, "  ROC_AUC:", roc_auc)

Кол-во итераций: 500   F1: 0.2679   ROC_AUC: 0.7455
Кол-во итераций: 20500   F1: 0.2679   ROC_AUC: 0.7455
Кол-во итераций: 40500   F1: 0.2679   ROC_AUC: 0.7455
Кол-во итераций: 60500   F1: 0.2679   ROC_AUC: 0.7455
Кол-во итераций: 80500   F1: 0.2679   ROC_AUC: 0.7455
Кол-во итераций: 100500   F1: 0.2679   ROC_AUC: 0.7455
Кол-во итераций: 120500   F1: 0.2679   ROC_AUC: 0.7455
Кол-во итераций: 140500   F1: 0.2679   ROC_AUC: 0.7455
Кол-во итераций: 160500   F1: 0.2679   ROC_AUC: 0.7455
Кол-во итераций: 180500   F1: 0.2679   ROC_AUC: 0.7455
-лучший результат-
Кол-во итераций: 500   F1: 0.2679   ROC_AUC: 0.7455


Алгоритм не улучшает результатов при увеличении гиперпараметров. Вероятно этот алгоритм слабо подходит для решения задачи. Результаты ниже требуемых по ТЗ и ниже полученных моделью случайного леса.

Попробуем устранить разбалансировку классов, т.к. вероятно она снижает качество результатов.

## Борьба с дисбалансом
Посмотрим на балансировку класcов целевого параметра. Имеется явный перекос почти в 4 раза в пользу оставшихся клиентов, что является благом для банка, но не для модели обучения.

In [13]:
print(df.groupby('Exited')['Exited'].count())

Exited
0    7963
1    2037
Name: Exited, dtype: int64


Устраним балансировку с помощью увеличения количества признаков с исходом "1". Для этого воспользуемся функцией upsample. В результате получаем перемешанный и увеличенный в 3 раза признак с исходом "1". 

In [14]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 3)

Так же попробуем устранить разбалансировку путем уменьшения количества признаков с исходом "0". Для этого воспользуемся функцией downsample. В результате получаем перемешанный и уменьшенный в 3 раза датасет с исходом "0". 

In [15]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.33)

Так же балансировку можно устранить путем изменения параметров решателя с помощью изменения веса. Этот прием технически одинаков с upsampling.

## Обучение отбалансированной модели
### Обучение отбалансированной upsampling модели случайного леса
Обучим модель случайного леса на отбаласированном "вверх" датасете и посмотрим на результаты валидационной выборки.

In [16]:
# лучшие метрики
f1_best = 0
roc_auc_best=0
# лучшие параметры случайного леса
n_est_best = 0
n_depth_best = 0

for depth in range(5, 12, 2): 
    for est in range(300, 1000, 200):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth )
        model.fit(features_upsampled, target_upsampled)
        predicted_valid = model.predict(features_valid)
        f1 = round( f1_score(target_valid, predicted_valid),4)
        #вычисление roc_auc
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid),4)
        print("Кол-во деревьев:",est ,"  глубина:", depth,"  F1:", f1, "  ROC_AUC:", roc_auc)
        
        #ищем лучшую модель по количеству деревьев
        if f1>f1_best and roc_auc>roc_auc_best:
            f1_best = f1
            roc_auc_best = roc_auc
            n_est_best = est
            n_depth_best = depth
            model_fr_up_best = model

predicted_valid = model_fr_up_best.predict(features_valid)
f1 = round( f1_score(target_valid, predicted_valid), 4)
probabilities_valid = model_fr_up_best.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid),4)
print('-лучший результат-')
print("Кол-во деревьев:",n_est_best,"  глубина:", n_depth_best,"  F1:", f1, "  ROC_AUC:", roc_auc)

Кол-во деревьев: 300   глубина: 5   F1: 0.6228   ROC_AUC: 0.8461
Кол-во деревьев: 500   глубина: 5   F1: 0.6212   ROC_AUC: 0.8474
Кол-во деревьев: 700   глубина: 5   F1: 0.6228   ROC_AUC: 0.8474
Кол-во деревьев: 900   глубина: 5   F1: 0.6221   ROC_AUC: 0.8474
Кол-во деревьев: 300   глубина: 7   F1: 0.63   ROC_AUC: 0.8533
Кол-во деревьев: 500   глубина: 7   F1: 0.6284   ROC_AUC: 0.8534
Кол-во деревьев: 700   глубина: 7   F1: 0.6318   ROC_AUC: 0.8535
Кол-во деревьев: 900   глубина: 7   F1: 0.6302   ROC_AUC: 0.8539
Кол-во деревьев: 300   глубина: 9   F1: 0.6209   ROC_AUC: 0.8533
Кол-во деревьев: 500   глубина: 9   F1: 0.6238   ROC_AUC: 0.8532
Кол-во деревьев: 700   глубина: 9   F1: 0.6256   ROC_AUC: 0.8532
Кол-во деревьев: 900   глубина: 9   F1: 0.6223   ROC_AUC: 0.8532
Кол-во деревьев: 300   глубина: 11   F1: 0.6022   ROC_AUC: 0.8486
Кол-во деревьев: 500   глубина: 11   F1: 0.6118   ROC_AUC: 0.8488
Кол-во деревьев: 700   глубина: 11   F1: 0.6078   ROC_AUC: 0.8491
Кол-во деревьев: 900   г

Результаты метрики F1 превышают минимально требуемые по заданию. Метрика AUC_ROC на уровне не очень высоком. F1=0.627 ROC_AUC=0.77

### Обучение отбалансированной downsampling модели случайного леса
Обучим модель случайного леса на отбаласированном "вниз" датасете и посмотрим на результаты валидационной выборки.

In [17]:
# лучшие метрики
f1_best = 0
roc_auc_best=0
# лучшие параметры случайного леса
n_est_best = 0
n_depth_best = 0

for depth in range(5, 12, 2): 
    for est in range(300, 1000, 200):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth )
        model.fit(features_downsampled, target_downsampled)
        predicted_valid = model.predict(features_valid)
        f1 = round( f1_score(target_valid, predicted_valid),4)
        #вычисление roc_auc
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid),4)
        print("Кол-во деревьев:",est ,"  глубина:", depth,"  F1:", f1, "  ROC_AUC:", roc_auc)
        
        #ищем лучшую модель по количеству деревьев
        if f1>f1_best and roc_auc>roc_auc_best:
            f1_best = f1
            roc_auc_best = roc_auc
            n_est_best = est
            n_depth_best = depth
            model_fr_dw_best = model

predicted_valid = model_fr_dw_best.predict(features_valid)
f1 = round( f1_score(target_valid, predicted_valid), 4)
probabilities_valid = model_fr_dw_best.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid),4)
print('-лучший результат-')
print("Кол-во деревьев:",n_est_best,"  глубина:", n_depth_best,"  F1:", f1, "  ROC_AUC:", roc_auc)

Кол-во деревьев: 300   глубина: 5   F1: 0.6219   ROC_AUC: 0.845
Кол-во деревьев: 500   глубина: 5   F1: 0.618   ROC_AUC: 0.8458
Кол-во деревьев: 700   глубина: 5   F1: 0.6164   ROC_AUC: 0.8459
Кол-во деревьев: 900   глубина: 5   F1: 0.6156   ROC_AUC: 0.846
Кол-во деревьев: 300   глубина: 7   F1: 0.6246   ROC_AUC: 0.8502
Кол-во деревьев: 500   глубина: 7   F1: 0.625   ROC_AUC: 0.8508
Кол-во деревьев: 700   глубина: 7   F1: 0.629   ROC_AUC: 0.8507
Кол-во деревьев: 900   глубина: 7   F1: 0.6298   ROC_AUC: 0.8507
Кол-во деревьев: 300   глубина: 9   F1: 0.6211   ROC_AUC: 0.8495
Кол-во деревьев: 500   глубина: 9   F1: 0.6255   ROC_AUC: 0.8501
Кол-во деревьев: 700   глубина: 9   F1: 0.6235   ROC_AUC: 0.8503
Кол-во деревьев: 900   глубина: 9   F1: 0.6242   ROC_AUC: 0.8504
Кол-во деревьев: 300   глубина: 11   F1: 0.6192   ROC_AUC: 0.8477
Кол-во деревьев: 500   глубина: 11   F1: 0.6146   ROC_AUC: 0.8479
Кол-во деревьев: 700   глубина: 11   F1: 0.616   ROC_AUC: 0.8477
Кол-во деревьев: 900   глуби

### Обучение неотбалансированной модели с параметрами решателя
Изменение выборок не единственный метод устранить разбалансированность. С помощью гиперпараметров решателя можно придать больший вес значениям, которые редко появляются. 

In [18]:
# лучшие метрики
f1_best = 0
roc_auc_best=0
# лучшие параметры случайного леса
n_est_best = 0
n_depth_best = 0

for depth in range(5, 12, 2): 
    for est in range(300, 1000, 200):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth, class_weight='balanced')
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = round( f1_score(target_valid, predicted_valid),4)
        #вычисление roc_auc
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid),4)
        print("Кол-во деревьев:",est ,"  глубина:", depth,"  F1:", f1, "  ROC_AUC:", roc_auc)
        
        #ищем лучшую модель по количеству деревьев
        if f1>f1_best and roc_auc>roc_auc_best:
            f1_best = f1
            roc_auc_best = roc_auc
            n_est_best = est
            n_depth_best = depth
            model_fr_wt_best = model

predicted_valid = model_fr_wt_best.predict(features_valid)
f1 = round( f1_score(target_valid, predicted_valid), 4)
probabilities_valid = model_fr_wt_best.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]
roc_auc = round( roc_auc_score(target_valid, probabilities_one_valid), 4)
print('-лучший результат-')
print("Кол-во деревьев:",n_est_best,"  глубина:", n_depth_best,"  F1:", f1, "  ROC_AUC:", roc_auc)

Кол-во деревьев: 300   глубина: 5   F1: 0.6103   ROC_AUC: 0.8494
Кол-во деревьев: 500   глубина: 5   F1: 0.616   ROC_AUC: 0.8496
Кол-во деревьев: 700   глубина: 5   F1: 0.6142   ROC_AUC: 0.8492
Кол-во деревьев: 900   глубина: 5   F1: 0.6149   ROC_AUC: 0.8492
Кол-во деревьев: 300   глубина: 7   F1: 0.6157   ROC_AUC: 0.8538
Кол-во деревьев: 500   глубина: 7   F1: 0.6188   ROC_AUC: 0.8534
Кол-во деревьев: 700   глубина: 7   F1: 0.6197   ROC_AUC: 0.8534
Кол-во деревьев: 900   глубина: 7   F1: 0.6167   ROC_AUC: 0.8533
Кол-во деревьев: 300   глубина: 9   F1: 0.6227   ROC_AUC: 0.8541
Кол-во деревьев: 500   глубина: 9   F1: 0.628   ROC_AUC: 0.8541
Кол-во деревьев: 700   глубина: 9   F1: 0.6224   ROC_AUC: 0.854
Кол-во деревьев: 900   глубина: 9   F1: 0.6249   ROC_AUC: 0.854
Кол-во деревьев: 300   глубина: 11   F1: 0.5992   ROC_AUC: 0.8486
Кол-во деревьев: 500   глубина: 11   F1: 0.6018   ROC_AUC: 0.849
Кол-во деревьев: 700   глубина: 11   F1: 0.6005   ROC_AUC: 0.8493
Кол-во деревьев: 900   глуб

### Поиск лучшей модели по уровню метрик
Рассматривались разные модели, причем у моделей менялись гиперапараметры, что в большинстве случаев приводило к изменению показателей метрик. По результатам валидационной выборки получились следующие параметры:
- модель случайного дерева с дисбалансом..............F1: 0.5701...ROC_AUC: 0.8515
- модель логистической регрессии с дисбалансом....F1: 0.2716...ROC_AUC: 0.7456
- модель случайного леса с upsampling......................F1: 0.6302...ROC_AUC: 0.8538
- модель случайного леса с downsampling.................F1: 0.6251...ROC_AUC: 0.8503
- модель случайного леса с параметром balanced....F1: 0.6234...ROC_AUC: 0.8541


Среди всех полученных моделей наилучшими параметрами обладают модели случайного леса. Модель логистической регрессии показывает результаты ниже требуемых. Устранение дисбаланса позволило повысить метрики моделей с 0,57 до 0,62, что отвечает минимальным требованиям к модели. Среди сбалансированных моделей результаты отличиются незначительно, в пределах погрешности, однако самый лучший результат показал метод upsampling с результатами гиперпараметров: количество деревьев 900, глубина 7.

## Результаты на тестовой выборке
Для тестовой выборке используется тестовая выборка, которая ранее не использовалась. Для анализа будет использоваться модель случайного леса, полученная с upsampling методом устранения дисбаланса. 

In [19]:
predicted_test = model_fr_up_best.predict(features_test)
f1 = round( f1_score(target_test, predicted_test), 4)
probabilities_test = model_fr_up_best.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
roc_auc = round( roc_auc_score(target_test, probabilities_one_test), 4)
print("Результат на тестовой выборке:","  F1:", f1, "  ROC_AUC:", roc_auc)

Результат на тестовой выборке:   F1: 0.6046   ROC_AUC: 0.8529


Результаты на тестовой выборке оказались заметно хуже, чем на валидационной, но превышают минимально требуемые. 

## Заключение 


В данной работе на основе полученной информации был сформирован датасет для анализа. Для обучения модели произведены следующие подготовительные работы: удалены ненужные столбцы датасета, категориальные переменные были кодифицированы, значения чисел были нормализованны. Датасет был разбит на 3 части - для обучения, валидации и тестовую. На основе первых двух выборок были проверены модели случайного леса и логистической регрессии. Гиперпараметры моделей принимали различные значения в поисках максимизации метрик f1 и roc_auc. Полученные значения изначально оказались ниже требуемых. С целью повышения качества модели произведена балансировка значений некоторых столбцов тремя методами: upsampling, downsampling и параметрами решателя. В итоге была выбрана лучшая модель, которая на тестовой выборке показала результат F1=0.6023, ROC_AUC=0.85, что превышает минимально требуемый уровень. Модель готова к использованию и показывает удовлетворительные результаты. 